# SEC Anomaly Detection — Validation

This notebook provides a repeatable validation workflow for the detectors:
- NT filings
- Friday after-hours (Friday burying)
- 8-K monthly spike


In [ ]:
import json
from pathlib import Path

import pandas as pd
import sqlite3

db_path = Path.cwd().parent / 'data' / 'sec_anomaly.db'
print(f'Using DB: {db_path}')

def load_alerts(anomaly_type: str) -> pd.DataFrame:
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(
        'SELECT * FROM alerts WHERE anomaly_type = ? ORDER BY created_at DESC',
        conn,
        params=(anomaly_type,),
    )
    conn.close()
    if df.empty:
        return df
    df['details'] = df['details'].apply(json.loads)
    details_df = pd.json_normalize(df['details'])
    return pd.concat([df.drop(columns=['details']), details_df], axis=1)


In [ ]:
# Overall alert counts
conn = sqlite3.connect(db_path)
summary = pd.read_sql_query(
    'SELECT anomaly_type, COUNT(*) AS count FROM alerts GROUP BY anomaly_type',
    conn,
)
conn.close()
print(summary.to_string(index=False))


## NT Filings Validation

Sanity checks:
- Alerts should only contain `NT %` or `NT-%` filings.
- Spot-check a few alerts to confirm filing type.


In [ ]:
nt_alerts = load_alerts('NT_FILING')
if nt_alerts.empty:
    print('No NT alerts yet. Run src/detection/nt_detection.py first.')
else:
    cols = [
        'company_ticker',
        'company_name',
        'filing_type',
        'filed_at',
        'severity_score',
        'created_at',
    ]
    available = [c for c in cols if c in nt_alerts.columns]
    print(nt_alerts[available].head(15).to_string(index=False))


## Friday After-Hours Validation

Sanity checks:
- Filing time should be Friday after 4pm US/Eastern.
- Forms should be 8-K or 8-K/A (MVP scope).


In [ ]:
friday_alerts = load_alerts('FRIDAY_BURYING')
if friday_alerts.empty:
    print('No Friday burying alerts yet. Run src/detection/friday_detection.py first.')
else:
    cols = [
        'company_ticker',
        'company_name',
        'filing_type',
        'filed_at',
        'severity_score',
        'created_at',
    ]
    available = [c for c in cols if c in friday_alerts.columns]
    print(friday_alerts[available].head(15).to_string(index=False))


## 8-K Monthly Spike Validation

Sanity checks:
- Count should exceed baseline mean + 2σ.
- Baseline includes zero-months in the lookback window.


In [ ]:
spike_alerts = load_alerts('8K_SPIKE')
if spike_alerts.empty:
    print('No 8-K spike alerts yet. Run src/detection/8k_spike_detection.py first.')
else:
    cols = [
        'company_ticker',
        'company_name',
        'month',
        'count',
        'baseline_mean',
        'baseline_std',
        'threshold',
        'severity_score',
        'created_at',
    ]
    available = [c for c in cols if c in spike_alerts.columns]
    print(spike_alerts[available].head(20).to_string(index=False))

    print('Top spikes by count:')
    print(spike_alerts.sort_values('count', ascending=False)[available].head(10).to_string(index=False))


## Spot-Check Helper (per company)

Use this to inspect filing volume by month for a specific company.


In [ ]:
def show_company_8k_history(cik: int, months: int = 6) -> pd.DataFrame:
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(
        """
        SELECT filed_date
        FROM filing_events
        WHERE cik = ?
          AND filing_type IN ('8-K', '8-K/A')
          AND filed_date >= date('now', ?)
        """,
        conn,
        params=(cik, f'-{months} months'),
    )
    conn.close()

    if df.empty:
        print('No filings found for this CIK in the lookback window.')
        return df

    df['filed_date'] = pd.to_datetime(df['filed_date'], format='mixed', utc=True)
    monthly = (
        df.set_index('filed_date')
          .resample('MS')
          .size()
          .rename('count')
    )
    return monthly.reset_index()

# Example:
# show_company_8k_history(320193)
